Задание
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше


In [1]:
! pip install stop_words
! pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.probability import FreqDist

import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, MaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import EarlyStopping  
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder
import gensim

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#! pip install --upgrade xlrd

In [4]:
#Загрузка данных
df = pd.read_excel('/content/отзывы за лето.xls')
df.head(5)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [5]:
#Разбиваем на трейн и тест
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((13841, 3), (6818, 3))

In [6]:
#Предобрабатываем данные
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)


In [7]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [8]:
tokens = word_tokenize(train_corpus)
tokens[:5]

['наконецтый', 'исправить', 'чушь', 'снеоргинальный', 'прошивка']

In [9]:
# Фильтрация токенов
tokens_filtered = [word for word in tokens if word.isalnum()]

In [10]:
max_words = 800

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [11]:
tokens_filtered_top[:5]

['приложение', 'удобно', 'работать', 'удобный', 'отлично']

In [12]:
#Создаем словарь
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [13]:
#Преобразуем текст
max_len = 100

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [14]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)

In [15]:
# Проведем экнодинг лейблов

num_classes = df['Rating'].nunique()
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

In [16]:
y_train = to_categorical(train_enc_labels, num_classes)
y_test = to_categorical(test_enc_labels, num_classes)

**Обучаем нейронную сеть с предобученными векторами в слое Embedding**

In [17]:
!wget http://vectors.nlpl.eu/repository/20/180.zip

--2022-07-04 11:06:53--  http://vectors.nlpl.eu/repository/20/180.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 484452317 (462M) [application/zip]
Saving to: ‘180.zip’

180.zip             100%[===================>] 462.01M  12.9MB/s    in 39s     

2022-07-04 11:07:33 (11.9 MB/s) - ‘180.zip’ saved [484452317/484452317]



In [18]:
!unzip 180.zip

Archive:  180.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [19]:
#Загрузим предобученные векторы
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/content/model.bin', binary=True) .vectors 

In [20]:
#обрежем word_vectors по размеру эмбеддингов по количеству слов, которые у нас есть в словаре
word_vectors_emd = [word_vectors[i][:256] for i in range(max_words)]
word_vectors_emd[1]

array([ 0.56827927, -0.3209115 , -0.01912439, -1.9356475 , -0.8124491 ,
        1.657973  , -1.3159128 ,  1.1186373 , -2.0223322 , -3.2186239 ,
       -0.8087832 , -1.6359693 ,  0.30781016, -1.9637547 ,  1.1508301 ,
        0.09154594,  1.499977  , -1.9233339 , -1.395698  ,  0.4522272 ,
        1.3875449 , -0.6991994 ,  1.1241058 , -0.8872212 ,  1.5669478 ,
        0.10525192, -0.72891355, -1.7446259 , -0.39187282, -0.21783888,
        0.99993414, -2.8352168 ,  0.28585172, -1.2888701 , -0.7390094 ,
        0.7050435 ,  2.6338942 ,  0.37333074,  2.078776  , -0.19290456,
       -0.73848265, -1.7742666 ,  1.6497376 ,  3.7614036 , -1.2978705 ,
       -0.00728586,  1.3725481 , -1.5491337 , -0.81334364, -0.7748071 ,
       -0.61844414,  0.7076229 ,  0.53189814, -0.519196  ,  1.2863954 ,
        0.99337226,  1.1154282 , -0.930117  ,  1.572236  , -1.2262355 ,
       -0.86334157,  0.61980397, -0.09515494,  0.5440264 , -0.50054914,
        0.32205996, -1.49439   ,  0.63581914,  2.5604725 ,  0.62

In [21]:
#инициализируем веса в эмбеддинге
initializer = tf.keras.initializers.Constant(word_vectors_emd)

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=256, embeddings_initializer = initializer, input_length=max_len)) 
                    #inputdim -- размер словаря, outputdim -- длина вектора, input_length -- длина входной последовательности
                    #на вход: (батч, input_lenght), на выходе: (батч, input_lenght, output_dim)

model.add(Conv1D(64, 3, padding = 'same'))
                   #128 -- количество 1D-фильтров, длина фильтра -- 3
                   #на выходе (количество объектов, 128 - длина нового эмбединга)

model.add(MaxPool1D(2))  

model.add(Activation("relu"))


model.add(Conv1D(128, 3, padding = 'same'))

model.add(MaxPool1D(2))

model.add(Activation("relu"))
                    #применяем функцию активации к выходу предыдущего слоя

model.add(GlobalMaxPool1D())
                    # в каждой свертке для каждого элемента эмбединга оставляем максимум - получаем один 128-м эмбеддинг

model.add(Dense(128))
               #128-количество выходов
model.add(Activation("relu"))
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(num_classes))
                #num_classes = 5 -- количество выходов
model.add(Activation('softmax'))
                #преобразуем вектор в рапределение вероятностей

In [23]:
# Компилируем модель
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [24]:
# Обучаем модель без раней остановки
epochs = 10
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    # callbacks=[early_stopping]
                    )

Epoch 1/10
25/25 [==============================] - 16s 43ms/step - loss: 0.9359 - accuracy: 0.6917 - val_loss: 0.8109 - val_accuracy: 0.7199
Epoch 2/10
25/25 [==============================] - 0s 19ms/step - loss: 0.7606 - accuracy: 0.7433 - val_loss: 0.7599 - val_accuracy: 0.7300
Epoch 3/10
25/25 [==============================] - 0s 19ms/step - loss: 0.6931 - accuracy: 0.7631 - val_loss: 0.7633 - val_accuracy: 0.7372
Epoch 4/10
25/25 [==============================] - 0s 19ms/step - loss: 0.6496 - accuracy: 0.7754 - val_loss: 0.7337 - val_accuracy: 0.7408
Epoch 5/10
25/25 [==============================] - 0s 20ms/step - loss: 0.6026 - accuracy: 0.7933 - val_loss: 0.7838 - val_accuracy: 0.7350
Epoch 6/10
25/25 [==============================] - 0s 19ms/step - loss: 0.5686 - accuracy: 0.8091 - val_loss: 0.7593 - val_accuracy: 0.7372
Epoch 7/10
25/25 [==============================] - 0s 18ms/step - loss: 0.5116 - accuracy: 0.8303 - val_loss: 0.7699 - val_accuracy: 0.7379
Epoch 8/10
2

In [25]:
# Смотрим метрики
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test f1_score:', score[1])

14/14 [==============================] - 0s 14ms/step - loss: 0.8474 - accuracy: 0.7537


Test loss: 0.8473901152610779
Test f1_score: 0.7537400722503662


**Нейронная сеть с обучаемым слоем Embedding**


In [26]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=256, input_length=max_len)) 
                    #inputdim -- размер словаря, outputdim -- длина вектора, input_length -- длина входной последовательности
                    #на вход: (батч, input_lenght), на выходе: (батч, input_lenght, output_dim)

model.add(Conv1D(64, 3, padding = 'same'))
                   #128 -- количество 1D-фильтров, длина фильтра -- 3
                   #на выходе (количество объектов, 128 - длина нового эмбединга)

model.add(MaxPool1D(2))  

model.add(Activation("relu"))


model.add(Conv1D(128, 3, padding = 'same'))

model.add(MaxPool1D(2))

model.add(Activation("relu"))
                    #применяем функцию активации к выходу предыдущего слоя

model.add(GlobalMaxPool1D())
                    # в каждой свертке для каждого элемента эмбединга оставляем максимум - получаем один 128-м эмбеддинг

model.add(Dense(128))
               #128-количество выходов
model.add(Activation("relu"))
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(num_classes))
                #num_classes = 5 -- количество выходов
model.add(Activation('softmax'))
                #преобразуем вектор в рапределение вероятностей

In [27]:
#Компилируем модель
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [28]:
#Обучаем модель без раней остановки
epochs = 10
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    # callbacks=[early_stopping]
                    )

Epoch 1/10
25/25 [==============================] - 1s 29ms/step - loss: 1.1729 - accuracy: 0.6801 - val_loss: 0.9825 - val_accuracy: 0.7011
Epoch 2/10
25/25 [==============================] - 0s 17ms/step - loss: 0.8250 - accuracy: 0.7202 - val_loss: 0.7170 - val_accuracy: 0.7509
Epoch 3/10
25/25 [==============================] - 0s 17ms/step - loss: 0.6644 - accuracy: 0.7687 - val_loss: 0.7001 - val_accuracy: 0.7531
Epoch 4/10
25/25 [==============================] - 0s 18ms/step - loss: 0.6273 - accuracy: 0.7793 - val_loss: 0.7025 - val_accuracy: 0.7523
Epoch 5/10
25/25 [==============================] - 1s 22ms/step - loss: 0.6072 - accuracy: 0.7858 - val_loss: 0.7037 - val_accuracy: 0.7480
Epoch 6/10
25/25 [==============================] - 1s 20ms/step - loss: 0.5921 - accuracy: 0.7936 - val_loss: 0.7056 - val_accuracy: 0.7487
Epoch 7/10
25/25 [==============================] - 0s 17ms/step - loss: 0.5745 - accuracy: 0.7994 - val_loss: 0.7144 - val_accuracy: 0.7523
Epoch 8/10
25

In [29]:
#Смотрим метрики
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test f1_score:', score[1])

14/14 [==============================] - 0s 8ms/step - loss: 0.7121 - accuracy: 0.7686


Test loss: 0.7121235132217407
Test f1_score: 0.7685538530349731


**Вывод:** нейросеть, в которой мы обучали эмбеддинги а не использовали изначально инициализированные, показала лучший результат